# V5 of cleaning data
- drop any column with 50% or more nans
- still use ordinal data, 
- changed ordinal ranking scores
- drop zipcd,
- clean src_div_id, 
- no normalization for tree-based methods

In [1]:
import sys
print("Python version")
print (sys.version)

Python version
3.6.13 |Anaconda, Inc.| (default, Jun  4 2021, 14:25:59) 
[GCC 7.5.0]


In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

## Load training set

In [4]:
train_df = pd.read_csv('data/2021_Competition_Training.csv', low_memory=False)
train_df = train_df.drop('Unnamed: 0', axis=1)
train_df = train_df.set_index('ID', drop=True)
train_df = train_df.replace({'*': np.nan})

In [5]:
y_name = ['covid_vaccination']

In [6]:
# put y in the first column
cols = list(train_df.columns)
cols.remove('covid_vaccination')
cols = y_name + cols

In [7]:
train_df = train_df[cols]

In [8]:
train_df.shape

(974842, 366)

In [9]:
train_df['covid_vaccination'].value_counts()

no_vacc    805389
vacc       169453
Name: covid_vaccination, dtype: int64

## Drop columns with >=50% nan

In [10]:
# check na numbers
for i in np.arange(0, 1, 0.1):
    print(f'{i:.1f}', train_df.dropna(axis=1, how='any', thresh=int(train_df.shape[0]*i)).shape)

0.0 (974842, 366)
0.1 (974842, 366)
0.2 (974842, 366)
0.3 (974842, 365)
0.4 (974842, 364)
0.5 (974842, 364)
0.6 (974842, 364)
0.7 (974842, 362)
0.8 (974842, 340)
0.9 (974842, 335)


In [10]:
for col in train_df.columns:
    if train_df[col].count() < int(train_df.shape[0]*0.5):
        print(col, f'has {(train_df.shape[0]-train_df[col].count())/train_df.shape[0]} nans')

mabh_seg has 0.649258033609549 nans
lang_spoken_cd has 0.7421182099253008 nans


In [11]:
# drop columns with > 50% na
train_df = train_df.dropna(axis=1, how='any', thresh=int(train_df.shape[0]*0.5))

In [12]:
train_df.shape

(974842, 364)

## Drop cols with only one value

In [13]:
for name in train_df.columns:
    if train_df[name].nunique() == 1:
        train_df.drop(name, axis=1, inplace=True)
train_df.shape

(974842, 310)

## Drop zipcd (since they were artificially generated)

In [14]:
train_df = train_df.drop('zip_cd', axis=1)

In [15]:
train_df.shape

(974842, 309)

## Find numerical/categorical features

In [16]:
cd_set = []
for name in train_df.columns.values:
    if '_cd' in name:
        print(name)
        cd_set.append(name)

cms_orig_reas_entitle_cd
sex_cd
race_cd


In [17]:
id_set = []
for name in train_df.columns.values:
    if '_id' in name:
        print(name)
        id_set.append(name)

src_div_id


In [18]:
categorical_features = ( set(train_df.columns) - set(train_df.describe().columns) - set(y_name) ) | set(cd_set)
len(categorical_features)

86

In [19]:
for col in categorical_features:
    train_df[col] = train_df[col].astype(float, errors='ignore')

In [20]:
categorical_features = ( set(train_df.columns) - set(train_df.describe().columns) - set(y_name) ) | set(cd_set)
len(categorical_features)

52

In [21]:
train_df['src_div_id'] = train_df['src_div_id'].fillna(0).astype(int, errors='ignore')

In [22]:
train_df['covid_vaccination'] = train_df['covid_vaccination'].map({'vacc': 1, 'no_vacc': 0})

## Add missing values

In [23]:
# how many columns with missing values
train_df.isnull().any().sum()

181

In [24]:
# Use mean or mode
for name in train_df.columns:
    if train_df[name].hasnans:
        if name in categorical_features:
            print('categorical:', name)
            train_df[name].fillna(train_df[name].mode()[0], inplace=True)
        elif train_df[name].dtypes == float:
            print('numerical:', name)
            train_df[name].fillna(train_df[name].mean(), inplace=True)
        else:
            print(name)

numerical: atlas_pct_laccess_child15
numerical: atlas_recfacpth14
numerical: atlas_pct_fmrkt_frveg16
numerical: atlas_pct_free_lunch14
numerical: cons_chmi
categorical: mcc_ano_pmpm_ct_t_9-6-3m_b4
numerical: cons_ltmedicr
numerical: rx_gpi4_6110_pmpm_ct
numerical: atlas_pc_snapben15
numerical: credit_bal_nonmtgcredit_60dpd
numerical: atlas_pct_laccess_nhna15
numerical: credit_hh_nonmtgcredit_60dpd
numerical: rx_bh_pmpm_ct_0to3m_b4
numerical: cons_lwcm10
numerical: atlas_fsrpth14
numerical: auth_3mth_dc_home
numerical: atlas_wicspth12
categorical: rx_gpi2_17_pmpm_cost_t_12-9-6m_b4
numerical: cons_hxmioc
numerical: atlas_ghveg_farms12
numerical: credit_hh_bankcardcredit_60dpd
numerical: total_outpatient_allowed_pmpm_cost_6to9m_b4
numerical: cons_cwht
numerical: atlas_netmigrationrate1016
numerical: atlas_pct_laccess_snap15
numerical: atlas_retirement_destination_2015_upda
numerical: atlas_naturalchangerate1016
numerical: atlas_pct_laccess_hisp15
numerical: auth_3mth_dc_no_ref
numerical: 

In [25]:
# how many columns with missing values after filling na
train_df.isnull().any().sum()

0

In [26]:
# save cleaned dataset without missing value
train_df.to_csv('data/training_removed_missing_v5.csv')

In [27]:
train_df.shape

(974842, 309)

## Transform ordinal features

In [28]:
# to restart only
if 0:
    train_df = pd.read_csv('data/training_removed_missing_v5.csv', index_col='ID')

In [29]:
ordered_cat_set = set([])
ordered_cat = []
for name in categorical_features:
    if 'No Activity' in train_df[name].unique():
        ordered_cat_set = ordered_cat_set | set(train_df[name].unique())
        ordered_cat.append(name)
print(ordered_cat_set)

/home/ywang/miniconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.


{'No Activity', 'No_Change', 'Inc_1x-2x', 'Dec_1x-2x', 'Inc_4x-8x', 'New', 'Dec_4x-8x', 'Resolved', 'Dec_2x-4x', 'Dec_over_8x', 'Inc_over_8x', 'Inc_2x-4x'}


In [30]:
len(ordered_cat)

45

In [31]:
len(ordered_cat_set)

12

In [32]:
train_df[ordered_cat].head()

,rx_nonotc_pmpm_cost_t_6-3-0m_b4,rx_mail_net_paid_pmpm_cost_t_6-3-0m_b4,rx_nonbh_pmpm_ct_t_9-6-3m_b4,bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4,rx_nonmail_dist_gpi6_pmpm_ct_t_9-6-3m_b4,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,rej_med_er_net_paid_pmpm_cost_t_9-6-3m_b4,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4,rx_generic_pmpm_cost_t_6-3-0m_b4,total_allowed_pmpm_cost_t_9-6-3m_b4,total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4,total_physician_office_mbr_resp_pmpm_cost_t_9-6-3m_b4,med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4,rx_nonbh_pmpm_cost_t_9-6-3m_b4,rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4,rx_maint_pmpm_cost_t_12-9-6m_b4,total_ip_maternity_net_paid_pmpm_cost_t_12-9-6m_b4,rx_nonbh_net_paid_pmpm_cost_t_6-3-0m_b4,med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4,rx_gpi2_62_pmpm_cost_t_9-6-3m_b4,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4,rx_nonmaint_dist_gpi6_pmpm_ct_t_12-9-6m_b4,rx_maint_pmpm_cost_t_6-3-0m_b4,rx_maint_pmpm_ct_t_6-3-0m_b4,rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4,bh_ip_snf_admit_days_pmpm_t_9-6-3m_b4,rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4,total_med_net_paid_pmpm_cost_t_6-3-0m_b4,med_ambulance_coins_pmpm_cost_t_9-6-3m_b4,rx_gpi2_39_pmpm_cost_t_6-3-0m_b4,med_outpatient_mbr_resp_pmpm_cost_t_9-6-3m_b4,rx_overall_gpi_pmpm_ct_t_6-3-0m_b4,rx_branded_pmpm_ct_t_6-3-0m_b4,med_outpatient_visit_ct_pmpm_t_12-9-6m_b4,rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4,oontwk_mbr_resp_pmpm_cost_t_6-3-0m_b4,rx_mail_mbr_resp_pmpm_cost_t_9-6-3m_b4,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4,total_bh_copay_pmpm_cost_t_9-6-3m_b4,mcc_chf_pmpm_ct_t_9-6-3m_b4,rx_overall_gpi_pmpm_ct_t_12-9-6m_b4,mcc_ano_pmpm_ct_t_9-6-3m_b4
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1MObcfaSTac85Lca0Y8bbA6I,Dec_1x-2x,Inc_1x-2x,Inc_1x-2x,No Activity,Inc_1x-2x,No Activity,No Activity,No Activity,Inc_1x-2x,Dec_1x-2x,No Activity,No Activity,No Activity,No Activity,Dec_1x-2x,No_Change,Inc_1x-2x,No Activity,Dec_1x-2x,No Activity,No Activity,No Activity,No Activity,Dec_1x-2x,Dec_1x-2x,Dec_1x-2x,Dec_2x-4x,Inc_1x-2x,No Activity,Dec_1x-2x,No Activity,No Activity,No_Change,No Activity,Dec_1x-2x,No Activity,No Activity,No Activity,No Activity,Inc_1x-2x,No Activity,No Activity,No Activity,Dec_1x-2x,No Activity
5M89OSTL580dYeA849d3480I,Inc_1x-2x,Inc_1x-2x,New,No_Change,No Activity,No Activity,No Activity,No Activity,No Activity,Inc_over_8x,No Activity,No Activity,No Activity,No Activity,New,New,No Activity,No_Change,Inc_1x-2x,No Activity,No Activity,No Activity,No Activity,Inc_1x-2x,Dec_1x-2x,Inc_1x-2x,New,New,No Activity,Inc_1x-2x,No Activity,No Activity,Dec_1x-2x,No Activity,Inc_1x-2x,No Activity,No_Change,No Activity,No Activity,New,No Activity,No Activity,No Activity,No Activity,No Activity
MdOS23TLe18Y60043Acfa2I9,Inc_1x-2x,New,New,No Activity,Inc_1x-2x,No Activity,No Activity,No Activity,Dec_2x-4x,Inc_1x-2x,Dec_4x-8x,Dec_2x-4x,Dec_2x-4x,No Activity,New,No Activity,Dec_2x-4x,No Activity,Inc_2x-4x,Dec_4x-8x,No Activity,Inc_1x-2x,No Activity,No_Change,No_Change,Inc_1x-2x,Dec_1x-2x,Inc_1x-2x,No Activity,Inc_1x-2x,Inc_1x-2x,No Activity,No Activity,Dec_2x-4x,Inc_1x-2x,No Activity,Dec_4x-8x,No Activity,Dec_1x-2x,No_Change,No Activity,No Activity,No Activity,No_Change,No Activity
2ccMO510abSaT79cLfaYAIe4,Dec_4x-8x,Resolved,Dec_1x-2x,No Activity,Dec_4x-8x,No Activity,No Activity,No Activity,Inc_2x-4x,New,Resolved,No Activity,No Activity,No Activity,Dec_1x-2x,No Activity,Inc_1x-2x,No Activity,Dec_over_8x,No Activity,No Activity,No Activity,No Activity,Dec_2x-4x,Dec_1x-2x,Dec_2x-4x,Dec_1x-2x,Dec_1x-2x,No Activity,No_Change,No Activity,No Activity,No Activity,No Activity,Dec_1x-2x,No Activity,No Activity,No Activity,No Activity,Inc_1x-2x,No Activity,No Activity,No Activity,Inc_1x-2x,No Activity
0M9811Ocb1ST94LY3f5A9I00,Dec_1x-2x,Dec_1x-2x,Inc_1x-2x,No Activity,No Activity,Resolved,No Activity,No Activity,Inc_1x-2x,Dec_1x-2x,No Activity,No Activity,No Activi

### Set ordinal scores

These trend variables compare the numerator to the denominator, so 0-3-6 compares 3 months prior to score date (0-3, numerator) to the 3-6 months prior to the score date (3-6, denominator).  
For 3-6-9, it’s 3-6 months prior to score date as numerator and 6-9 months prior to score date as denominator.  
We’ll use numerator and denominator to help explain things below.  
- Is there a difference between the values of "Not_Activity" and "No_Change"?
No Activity = No claims in numerator or denominator; No change = claims in both numerator and denominator, but ratio between 0.99 and 1.01
- What does the value of "Resolved" mean?
Resolved = Had claims in denominator, but none in numerator
- We are assuming "New" means the patient had the cost incurred for the first time, is that a correct assumption?
New = had claims in numerator, but not denominator


In [33]:
ordered_cat_transform_tab = {'Inc_over_8x': 8,
                             'Inc_4x-8x': 6,
                             'Inc_2x-4x': 3,
                             'Inc_1x-2x': 1.5,
                             'No_Change': 0,
                             'Dec_1x-2x': -1.5,
                             'Dec_2x-4x': -3,
                             'Dec_4x-8x': -6,
                             'Dec_over_8x': -8,
                             'Resolved': -10, 
                             'No Activity': 0.5,
                             'New': 10}

In [34]:
# check trend
train_df.groupby('rx_gpi2_62_pmpm_cost_t_9-6-3m_b4')[y_name].mean().sort_values(y_name)

,covid_vaccination
rx_gpi2_62_pmpm_cost_t_9-6-3m_b4,
Dec_4x-8x,0.162500
Resolved,0.162727
No Activity,0.172730
New,0.192548
Inc_2x-4x,0.202619
Dec_over_8x,0.212121
Inc_4x-8x,0.218935
Inc_1x-2x,0.226991
Dec_2x-4x,0.235988


In [35]:
for name in ordered_cat:
    train_df[name] = train_df[name].map(ordered_cat_transform_tab)

## Transform categorical features

In [36]:
categorical_features = (set(categorical_features)-set(ordered_cat)) | set(['src_div_id'])
categorical_features

{'cms_orig_reas_entitle_cd',
 'cons_hhcomp',
 'cons_mobplus',
 'hedis_dia_hba1c_ge9',
 'hum_region',
 'race_cd',
 'sex_cd',
 'src_div_id'}

In [37]:
train_df[categorical_features].sample(20)

,cons_hhcomp,race_cd,cons_mobplus,src_div_id,cms_orig_reas_entitle_cd,hedis_dia_hba1c_ge9,sex_cd,hum_region
ID,,,,,,,,
40M99fO294Sd1eT2LY9dA15I,A,1.0,P,0,0.0,N,M,TEXAS
5fMO5Sb3T3a3Lbfd4eYfA54I,L,1.0,U,0,0.0,N,F,TEXAS
MO20S751T3654L42Y25AId89,L,1.0,S,0,1.0,N,F,TEXAS
MO9a2d96eS2ddTf1L12YAI74,A,5.0,S,0,1.0,N,M,FLORIDA
0dMOadf0STLd1a5aY5fAd9Ib,J,1.0,U,0,1.0,N,M,MID-ATLANTIC/NORTH CAROLINA
eM65aO8S588TfL7Y9e38cAfI,B,1.0,M,0,0.0,N,F,GREAT LAKES/CENTRAL NORTH
fMbd3O18eSdcT8LYfa7A7I73,L,1.0,M,0,0.0,N,F,INTERMOUNTAIN
MO200S96cTL857aY9Adf4I4b,A,1.0,M,0,0.0,N,F,CENTRAL WEST
McO9STc3aL6abfYd9d5A7I5a,J,4.0,M,0,0.0,N,M,CENTRAL WEST


In [38]:
train_df[id_set].value_counts()

0    898148
1     69291
2      5536
3       841
5       526
4       500
Name: src_div_id, dtype: int64

In [40]:
for name in categorical_features:
    train_df = pd.concat([train_df.drop(name, axis=1), pd.get_dummies(train_df[name], prefix=name, dummy_na=True)], axis=1)
    print(name, train_df.shape)

cons_hhcomp (974842, 322)
race_cd (974842, 329)
cons_mobplus (974842, 333)
src_div_id (974842, 339)
cms_orig_reas_entitle_cd (974842, 343)
hedis_dia_hba1c_ge9 (974842, 345)
sex_cd (974842, 347)
hum_region (974842, 363)


In [41]:
train_df.shape

(974842, 363)

## Save cleaned dataset

In [42]:
train_df.head()

,covid_vaccination,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ncal_pmpm_ct,total_bh_copay_pmpm_cost_t_9-6-3m_b4,cons_chmi,mcc_ano_pmpm_ct_t_9-6-3m_b4,rx_maint_pmpm_cost_t_12-9-6m_b4,cons_ltmedicr,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,credit_bal_nonmtgcredit_60dpd,rx_bh_mbr_resp_pmpm_cost_9to12m_b4,rx_nonbh_pmpm_cost_t_9-6-3m_b4,atlas_pct_laccess_nhna15,credit_hh_nonmtgcredit_60dpd,rx_bh_pmpm_ct_0to3m_b4,cons_lwcm10,atlas_fsrpth14,auth_3mth_dc_home,atlas_wicspth12,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,cons_hxmioc,rx_generic_pmpm_cost_t_6-3-0m_b4,cmsd2_sns_digest_abdomen_pmpm_ct,atlas_ghveg_farms12,credit_hh_bankcardcredit_60dpd,total_outpatient_allowed_pmpm_cost_6to9m_b4,cons_cwht,atlas_netmigrationrate1016,atlas_pct_laccess_snap15,bh_ncdm_ind,rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4,atlas_retirement_destination_2015_upda,rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4,atlas_naturalchangerate1016,ccsp_236_pct,rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4,atlas_pct_laccess_hisp15,auth_3mth_dc_no_ref,rx_overall_mbr_resp_pmpm_cost,rx_overall_gpi_pmpm_ct_0to3m_b4,auth_3mth_dc_snf,rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4,atlas_pct_laccess_hhnv15,auth_3mth_acute_end,auth_3mth_psychic,atlas_hiamenity,auth_3mth_bh_acute,credit_bal_consumerfinance,auth_3mth_acute_chf,rx_overall_gpi_pmpm_ct_t_6-3-0m_b4,rwjf_uninsured_pct,mcc_chf_pmpm_ct_t_9-6-3m_b4,rx_mail_mbr_resp_pmpm_cost_0to3m_b4,bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4,atlas_pct_wic15,ccsp_193_pct,auth_3mth_dc_hospice,atlas_pct_fmrkt_baked16,rx_nonmaint_mbr_resp_pmpm_cost,auth_3mth_acute_skn,atlas_veg_farms12,atlas_vlfoodsec_13_15,rx_gpi2_34_dist_gpi6_pmpm_ct,bh_ip_snf_net_paid_pmpm_cost,credit_hh_bankcard_severederog,rx_hum_16_pmpm_ct,est_age,rx_maint_pmpm_cost_t_6-3-0m_b4,cnt_cp_webstatement_pmpm_ct,atlas_pct_laccess_seniors15,phy_em_px_pct,atlas_percapitainc,rwjf_uninsured_adults_pct,rx_generic_mbr_resp_pmpm_cost_0to3m_b4,rwjf_air_pollute_density,rx_gpi2_02_pmpm_cost,atlas_recfac14,lab_albumin_loinc_pmpm_ct,atlas_pct_obese_adults13,rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4,rev_pm_obsrm_pmpm_ct,atlas_pct_sfsp15,total_physician_office_net_paid_pmpm_cost_9to12m_b4,atlas_pc_dirsales12,med_ip_snf_admit_days_pmpm,rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4,cms_tot_partd_payment_amt,rx_nonotc_dist_gpi6_pmpm_ct,rx_nonmaint_pmpm_ct,rx_nonbh_mbr_resp_pmpm_cost_6to9m_b4,cons_stlnindx,atlas_hipov_1115,rx_nonbh_mbr_resp_pmpm_cost,atlas_redemp_snaps16,atlas_berry_farms12,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,rwjf_inactivity_pct,rx_gpi2_72_pmpm_ct_6to9m_b4,cons_n2pmr,med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_res,rev_cms_ct_pmpm_ct,atlas_foodhub16,total_physician_office_copay_pmpm_cost,auth_3mth_acute_dig,auth_3mth_dc_acute_rehab,atlas_pct_fmrkt_anmlprod16,credit_num_agencyfirstmtg,total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4,atlas_type_2015_mining_no,atlas_agritrsm_rct12,rx_days_since_last_script,atlas_pct_laccess_pop15,auth_3mth_post_acute_res,auth_3mth_acute_inf,rx_gpi2_01_pmpm_cost_0to3m_b4,atlas_povertyallagespct,rwjf_uninsured_child_pct,rx_branded_pmpm_ct_t_6-3-0m_b4,med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4,credit_bal_mtgcredit_new,atlas_low_employment_2015_update,atlas_pct_diabetes_adults13,atlas_pct_laccess_nhasian15,atlas_deep_pov_all,atlas_net_international_migration_rate,atlas_deep_pov_children,bh_ncdm_pct,auth_3mth_non_er,atlas_foodinsec_child_03_11,rx_branded_mbr_resp_pmpm_cost,atlas_pc_wic_redemp12,rwjf_mv_deaths_rate,auth_3mth_acute_cad,atlas_pct_reduced_lunch14,cons_nwperadult,total_allowed_pmpm_cost_t_9-6-3m_b4,rx_hum_28_pmpm_cost,atlas_totalocchu,med_physician_office_ds_clm_6to9m_b4,atlas_pct_loclfarm12,rx_generic_mbr_resp_pmpm_cost,total_outpatient_mbr_resp_pmpm_cost_6to9m_b4,rx_gpi4_3400_pmpm_ct,lab_dist_loinc_pmpm_ct,atlas_pct_nslp15,rx_generic_pmpm_ct_0to3m_b4,oontwk_mbr_resp_pmpm_cost_t_6-3-0m_b4,atlas_pct_laccess_lowi15,bh_ncal_ind,atlas_pct_fmrkt_sfmnp16,rx_nonm

In [43]:
train_df.to_csv('data/training_cleaned_v5.csv')